In [7]:
from fastai.vision import *
from models import Darknet
from train import hyp
from utils import torch_utils
from utils.utils import compute_loss, build_targets


In [8]:
# https://github.com/cedrickchee/knowledge/blob/master/courses/fast.ai/deep-learning-part-2/2018-edition/lesson-8-object-detection.md
coco = untar_data(URLs.COCO_TINY)
voc2007 = untar_data(URLs.PASCAL_2007)
voc2012 = untar_data(URLs.PASCAL_2012)

In [9]:
# -------- TODO: kill COCO
images, lbl_bbox = get_annotations(coco/'train.json')
img2bbox = dict(zip(images, lbl_bbox))
get_y_func = lambda o:img2bbox[o.name]

In [10]:
# https://pjreddie.com/darknet/yolo/#train-voc
files = [
    voc2007 / 'train.json', 
    voc2007 / 'valid.json',
    voc2007 / 'test.json',
    voc2012 / 'train.json', 
    voc2012 / 'valid.json'
    ]
jsons = [(it, json.load(it.open())) for it in files]
images = [{**img, 'file': fn} for (fn, json) in jsons for img in json["images"]]
images = { i["id"] : i for i in images }
annotations = [item for (fn, json) in jsons for item in json["annotations"]]
f"{len(images)}; {len(annotations)}"

'21503; 62199'

In [11]:
for anno in annotations:
    image = images[anno['image_id']]
    image.setdefault('annotations', []).append(anno)


In [12]:
person_cat = [it for it in jsons[0][1]["categories"] if it["name"] == "person"][0]["id"]
person_cat

15

In [13]:
def has_person(img): 
    return [] != [a for a in img['annotations'] if a['category_id'] == person_cat]
positive_samp = [img for img in images.values() if has_person(img)]
negative_samp = [img for img in images.values() if not has_person(img)]
f"{len(positive_samp)}; {len(negative_samp)}"

'8566; 12937'

In [14]:
negative_samp = negative_samp[:len(positive_samp)]
len(negative_samp)

8566

In [15]:
samples = positive_samp + negative_samp

In [16]:
def get_folder(f):
    if 'train' in str(f) or 'valid' in str(f): return 'train'
    return 'test'
def make_path(p):
    return p['file'].parent / get_folder(p['file']) / p['file_name']
posix_paths = [make_path(p) for p in samples]
# lst = ImageList(posix_paths)
lst = ObjectItemList(posix_paths)

In [17]:
def split_func(sample):
    return '2007' in str(sample['file'].parent) and 'test' in str(sample['file'])
valid = set()
for sample in samples:
    if split_func(sample):
        valid.add(make_path(sample))
len(valid)

4952

In [27]:
def filename_to_key(filename):
    trunk = os.path.splitext(filename)[0]
    txt = trunk.replace("_", "")
    id = int(txt)
    return id
def get_y_func(path):
    key = filename_to_key(path.name)
    image = images[key]
    return image['annotations']

In [28]:
data = (lst
        .split_by_valid_func(lambda it: it in valid)                          
        .label_from_func(get_y_func)
        #How to find the labels? -> use get_y_func on the file name of the data
        .transform(get_transforms(), tfm_y=True)
        #Data augmentation? -> Standard transforms; also transform the label images
        .databunch(bs=16, collate_fn=bb_pad_collate))   
        #Finally we convert to a DataBunch, use a batch size of 16,
        # and we use bb_pad_collate to collate the data into a mini-batch

IndexError: list index out of range

In [ ]:
data.show_batch(rows=2, ds_type=DatasetType.Valid, figsize=(6,6))


In [ ]:
device = 'cpu'
arc = 'default'
cfg = 'cfg/yolov3-tiny-anchors.cfg'
device = torch_utils.select_device(device, apex=False, batch_size=64)
model = Darknet(cfg, arc=arc).to(device)

In [ ]:
def loss_func(predicted, boxes, classes):
    targets = []
    bs = classes.shape[0]
    max_detections = classes.shape[1]
    for img_idx in range(bs):
        for detect_idx in range(max_detections):
            clazz = classes[img_idx, detect_idx]
            if clazz == 0: continue
            x, y, w, h = boxes[img_idx, detect_idx]
            targets.append([img_idx, float(clazz), float(x), float(y), float(w), float(h)])
    targets = tensor(targets)
    targets = build_targets(model, targets)
    loss = compute_loss(predicted, targets, model)
    return loss

In [ ]:
learner = Learner(data, model, loss_func=loss_func)

In [ ]:
model.arc = 'default'
model.nc = 7  # num classes
model.hyp = hyp
learner.fit(1)